In [1]:
import keras
import numpy as np
from keras import datasets
from numpy.linalg import eig
import matplotlib.pyplot as plt
from numpy import cov
from keras.datasets import mnist
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.svm import LinearSVC

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def reduce_dimensions_pca(x_train):
    mean = np.mean(x_train, axis=0)
    centered = x_train - mean

    cov_mat = cov(centered.T)
    eig_val, eig_vec = eig(cov_mat)
    eig_pairs = [(np.abs(eig_val[i]), eig_vec[:, i]) for i in range(len(eig_val))]

    # Sort the (eigenvalue, eigenvector) tuples from high to low
    eig_pairs.sort(key=lambda x: x[0], reverse=True)
    eig_val_sorted = np.array([x[0] for x in eig_pairs])
    eig_vec_sorted = np.array([x[1] for x in eig_pairs]).T
    return mean, eig_val_sorted, eig_vec_sorted

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(60000, 28*28)
x_test = x_test.reshape((10000, 28*28))
x_train = x_train/ 255
x_test = x_test / 255
mean, eig_val, eig_vec = reduce_dimensions_pca(x_train)

In [4]:
dimensions = [40, 80, 200]
c_param = [0.01 , 0.1 , 1, 10]
for dim in dimensions:
    for param in c_param:
        clf = LinearSVC(random_state=0, tol=1e-5, C=param)
        x_train_new = x_train - mean
        reduced = x_train_new.dot(eig_vec[:,:dim])
        clf.fit(reduced, y_train)
        x_test_new = x_test - mean
        x_test_proj = x_test_new.dot(eig_vec[:, :dim])
        pred = clf.predict(x_test_proj)
        print('c_param : {0} , dimensions {1}, acc {2}'.format(param, dim, accuracy_score(y_test, pred)))

c_param : 0.01 , dimensions 40, acc 0.8939
c_param : 0.1 , dimensions 40, acc 0.8946
c_param : 1 , dimensions 40, acc 0.895
c_param : 10 , dimensions 40, acc 0.893
c_param : 0.01 , dimensions 80, acc 0.9091
c_param : 0.1 , dimensions 80, acc 0.9102
c_param : 1 , dimensions 80, acc 0.9103
c_param : 10 , dimensions 80, acc 0.9012
c_param : 0.01 , dimensions 200, acc 0.9157
c_param : 0.1 , dimensions 200, acc 0.9183
c_param : 1 , dimensions 200, acc 0.9187
c_param : 10 , dimensions 200, acc 0.9123


# rbf kernel

In [5]:
dimensions = [40, 80, 200]
c_param = [0.01 , 0.1 , 1, 10]
for dim in dimensions:
    for param in c_param:
        clf = SVC(random_state=0, tol=1e-5, C=param, gamma = 0.1)
        x_train_new = x_train - mean
        reduced = x_train_new.dot(eig_vec[:,:dim])
        clf.fit(reduced, y_train)
        x_test_new = x_test - mean
        x_test_proj = x_test_new.dot(eig_vec[:, :dim])
        pred = clf.predict(x_test_proj)
        print('c_param : {0} , dimensions {1}, acc {2}'.format(param, dim, accuracy_score(y_test, pred)))

c_param : 0.01 , dimensions 40, acc 0.7625
c_param : 0.1 , dimensions 40, acc 0.9615
c_param : 1 , dimensions 40, acc 0.9821
c_param : 10 , dimensions 40, acc 0.9825
c_param : 0.01 , dimensions 80, acc 0.3489
c_param : 0.1 , dimensions 80, acc 0.9027
c_param : 1 , dimensions 80, acc 0.9783
c_param : 10 , dimensions 80, acc 0.9784
c_param : 0.01 , dimensions 200, acc 0.2837
c_param : 0.1 , dimensions 200, acc 0.7827
c_param : 1 , dimensions 200, acc 0.9673
c_param : 10 , dimensions 200, acc 0.9681


In [6]:
# Best results
dimensions = [40, 80, 200]
c_param = [5]
for dim in dimensions:
    for param in c_param:
        clf = SVC(random_state=0, tol=1e-5, C=param, gamma = 0.05)
        x_train_new = x_train - mean
        reduced = x_train_new.dot(eig_vec[:,:dim])
        clf.fit(reduced, y_train)
        x_test_new = x_test - mean
        x_test_proj = x_test_new.dot(eig_vec[:, :dim])
        pred = clf.predict(x_test_proj)
        print('c_param : {0} , dimensions {1}, acc {2}'.format(param, dim, accuracy_score(y_test, pred)))

c_param : 5 , dimensions 40, acc 0.9861
c_param : 5 , dimensions 80, acc 0.9867
c_param : 5 , dimensions 200, acc 0.9846
